In [7]:
"""
References: 
1) https://github.com/dogtagpki/pki/wiki/PKI-Certificate-Python-API
2) https://github.com/dogtagpki/pki/wiki/PKI-Certificate-Client-in-Python
3) https://frasertweedale.github.io/blog-redhat/posts/2014-06-02-dogtag-python-api.html

"""
from pki.client import PKIConnection
from pki.account import AccountClient
from pki.cert import CertClient
from pki.cert import CertSearchRequest


#connection = PKIConnection( protocol='http',hostname='localhost', port='8080') #this works
connection = PKIConnection( protocol='https',hostname='localhost', port='8443', verify=False)

connection.set_authentication_cert("/tmp/ca_admin_cert.pem")

cert_client = CertClient(connection)  

In [8]:
cert_search_request = CertSearchRequest(serial_from='6', status='VALID')
cert_data_infos = cert_client.list_certs(cert_search_request)
#cert_data_infos = cert_client.list_certs()


for cert_data_info in cert_data_infos:
    print("-------------------------  "+cert_data_info.serial_number+"    -----------------------------------")  
    print("Serial Number: " + cert_data_info.serial_number)
    print("Subject DN: " + cert_data_info.subject_dn)
    print("Status: " + cert_data_info.status)

-------------------------  0x1    -----------------------------------
Serial Number: 0x1
Subject DN: CN=CA Signing Certificate,OU=pki-tomcat,O=EXAMPLE
Status: VALID
-------------------------  0x2    -----------------------------------
Serial Number: 0x2
Subject DN: CN=CA OCSP Signing Certificate,OU=pki-tomcat,O=EXAMPLE
Status: VALID
-------------------------  0x3    -----------------------------------
Serial Number: 0x3
Subject DN: CN=pki.example.com,OU=pki-tomcat,O=EXAMPLE
Status: VALID
-------------------------  0x4    -----------------------------------
Serial Number: 0x4
Subject DN: CN=Subsystem Certificate,OU=pki-tomcat,O=EXAMPLE
Status: VALID
-------------------------  0x5    -----------------------------------
Serial Number: 0x5
Subject DN: CN=CA Audit Signing Certificate,OU=pki-tomcat,O=EXAMPLE
Status: VALID
-------------------------  0x6    -----------------------------------
Serial Number: 0x6
Subject DN: CN=PKI Administrator,E=caadmin@example.com,OU=pki-tomcat,O=EXAMPLE
Stat

In [10]:
# Create a dictionary that stores values required for certificate enrollment
inputs = {}

# Set the cert_request_type
inputs["cert_request_type"] = "pkcs10"

# Set the base64 encoded string of the crmf request generated to attribute cert_request
inputs['cert_request'] = "MIIBmDCCAQECAQAwWDELMAkGA1UEBhMCVVMxCzAJBgNVBAg" \
                             "MAk5DMRAwDgYDVQQHDAdSYWxlaWdoMRUwEwYDVQQKDAxSZW" \
                             "QgSGF0IEluYy4xEzARBgNVBAMMClRlc3RTZXJ2ZXIwgZ8wD" \
                             "QYJKoZIhvcNAQEBBQADgY0AMIGJAoGBAMJpWz92dSYCvWxl" \
                             "lrQCY5atPKCswUwyppRNGPnKmJ77AdHBBI4dFyET+h/+69j" \
                             "QMTLZMa8FX7SbyHvgbgLBP4Q/RzCSE2S87qFNjriOqiQCqJ" \
                             "mcrzDzdncJQiP+O7T6MSpLo3smLP7dK1Vd7vK0Vy8yHwV0e" \
                             "Bx7DgYedv2slBPHAgMBAAGgADANBgkqhkiG9w0BAQUFAAOB" \
                             "gQBvkxAGKwkfK3TKwLc5Mg0IWp8zGRVwxdIlghAL8DugNoc" \
                             "CNNgmZazglJOOehLuk0/NkLX1ZM5RrVgM09W6kcfWZtIwr5" \
                             "Uje2K/+6tW2ZTGrbizs7CNOTMzA/9H8CkHb4H9P/qRT275z" \
                             "HIocYj4smUnXLwWGsBMeGs+OMMbGvSrHg=="

#Set the values for requestor details
inputs['requestor_name'] = "Tester"
inputs['requestor_email'] = "example@redhat.com"

#enroll_req = cert_client.create_enrollment_request("caServerCert", inputs)
#req_infos = cert_client.submit_enrollment_request(enroll_req)

cert_data_objects = cert_client.enroll_cert('caServerCert', inputs)

PKIException: Unable to create enrollment request: Invalid Request

In [11]:
# Since the caUserCert profile is used in the example the cert_data_objects list contains only one element.
cert_data = cert_data_objects[0]

# Print the certificate information
print('Serial Number: ' + cert_data.serial_number)
print('Issuer: ' + cert_data.issuer_dn)
print('Subject: ' + cert_data.subject_dn)
print('Status: ' + cert_data.status)
print('Not Before: ' + cert_data.not_before)
print('Not After: ' + cert_data.not_after)
print('Encoded: ')
print(cert_data.encoded)
print("Pretty print format: ")
print(cert_data.pretty_print)

NameError: name 'cert_data_objects' is not defined

In [ ]:
"""
Summary:
* After the pre requisite dependencies were installed, I was able to invoke the Python API using the http protocol or https without ssl verification, to list all the certificates in the server
* When using https with SSL verification, I encountered an cerificate SSL verification error, which requires further investigation to resolve the issue. 
* It is likely that the certification enrollment related python API is also failing because of the SSL verification error.   

"""